# Segmenting Neighborhoods in Toronto, CA

#### Import pandas, install and import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
!pip install bs4

In [4]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

#### Scrape Wikipedia page for table and put it into Soup object

In [48]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [49]:
data = requests.get(url).text

In [50]:
soup = BeautifulSoup(data,"html5lib")

In [51]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Number of rows and columns in dataframe

In [52]:
df.shape

(103, 3)

### Adding Lat-Long Coordinates to Dataframe

In [53]:
lat_lng_coords = pd.read_csv("https://cocl.us/Geospatial_data")
lat_lng_coords = lat_lng_coords.sort_values(by='Postal Code',ascending=True)
lat_lng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Rename column in lat_lng_coords to facilitate merge with neighborhood dataframe

In [57]:
lat_lng_coords = lat_lng_coords.rename(columns={"Postal Code":"PostalCode"}) 
lat_lng_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [153]:
df2 = pd.merge(df,lat_lng_coords)
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Installing Geopy for geocoder

In [40]:
!conda install -c conda-forge geopy --yes 

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Clinton\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-2.1.0       

In [60]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [61]:
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Installing Folium 

In [43]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Clinton\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be DOWNGRADED:

  folium                                0.12.0-pyhd8ed1ab_1 --> 0.5.0-py_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [62]:
import folium # map rendering library

#### Creating a map of Toronto and adding neighborhood markers

In [63]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [65]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Examining the borough of Scarborough and its venues

In [155]:
scarborough_data = df2[df2['Borough'] == "Scarborough"].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [156]:
address = 'Scarborough, Toronto, Ontario'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


### Creating a map of Scarborough

In [157]:
# create map of Etobicoke using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Adding Foursquare credientials

In [73]:
CLIENT_ID = 'ZK0HJBVU3KJY1D5EFETMVUZ4R0HOO1TVHWOKWS0BWIGVDIKZ' # your Foursquare ID
CLIENT_SECRET = '10PXKLHCP5MR4DA1YWHN4JUJZAKXIOI3P2KQNTAXUG0100B3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZK0HJBVU3KJY1D5EFETMVUZ4R0HOO1TVHWOKWS0BWIGVDIKZ
CLIENT_SECRET:10PXKLHCP5MR4DA1YWHN4JUJZAKXIOI3P2KQNTAXUG0100B3


#### Creating a function that will retrieve all venues near each neighborhood

In [134]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [159]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],latitudes=scarborough_data['Latitude'],longitudes=scarborough_data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [160]:
print(scarborough_venues.shape)
scarborough_venues.head()

(91, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


We found 91 nearby venues

### Use One Hot Encoding to create Dataframe with all venue categories

In [161]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Station,...,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Station,...,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.00,0.000,0.000000,0.0,0.2500,0.000000,0.000000,...,0.0,0.000,0.000,0.0000,0.000000,0.25,0.000,0.0000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.00,0.000,0.000000,0.0,0.0000,0.000000,0.000000,...,0.0,0.000,0.000,0.0000,0.000000,0.25,0.000,0.0000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.00,0.125,0.125000,0.0,0.0000,0.000000,0.000000,...,0.0,0.000,0.000,0.0000,0.000000,0.00,0.000,0.0000,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.00,0.000,0.071429,0.0,0.0000,0.000000,0.000000,...,0.0,0.000,0.000,0.0000,0.071429,0.00,0.000,0.0000,0.071429,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.00,0.000,0.000000,0.0,0.0000,0.000000,0.000000,...,0.0,0.000,0.000,0.0000,0.000000,0.00,0.000,0.0000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.00,0.000,0.000000,0.0,0.0000,0.142857,0.000000,...,0.0,0.000,0.000,0.0000,0.000000,0.00,0.000,0.0000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.00,0.250,0.000000,0.0,0.0000,0.000000,0.125000,...,0.0,0.000,0.000,0.0000,0.000000,0.00,0.125,0.0000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.00,0.000,0.125000,0.0,0.1250,0.000000,0.000000,...,0.0,0.125,0.125,0.0000,0.000000,0.00,0.000,0.0000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.0,0.000,0.00,0.000,0.000000,0.0,0.0000,0.000000,0.166667,...,0.0,0.000,0.000,0.0000,0.000000,0.00,0.000,0.0000,0.000000,0.000000
9,"Malvern, Rouge",0.0,0.000,0.00,0.000,0.000000,0.0,0.0000,0.000000,0.000000,...,0.5,0.000,0.000,0.0000,0.000000,0.00,0.000,0.0000,0.000000,0.000000


Now we'll find the top 5 venue categories in each neighborhood

In [163]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.12
1      Thai Restaurant  0.12
2               Bakery  0.12
3                 Bank  0.12
4  Fried Chicken Joint  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                venue  freq
0            Pharmacy  0.14
1         Pizza Place  0.14
2  Chinese Restaurant  0.07
3        Noodle House  0.07
4         Gas Station  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                   venue  freq
0    American Restaurant   0.5
1               

In [164]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [170]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Gym Pool,Gym,Grocery Store,Gas Station
2,Cedarbrae,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Gas Station,Vietnamese Restaurant,Electronics Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Fast Food Restaurant,Convenience Store,Fried Chicken Joint,Gas Station,Bank
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Convenience Store,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


Use k-means clustering to find 5 unique neighborhood clusters

In [171]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how = 'right')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Gym Pool,Gym,Grocery Store,Gas Station
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Gas Station,Vietnamese Restaurant,Electronics Store
13,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,1,Pharmacy,Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Fast Food Restaurant,Convenience Store,Fried Chicken Joint,Gas Station,Bank
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Motel,Convenience Store,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [172]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

One of our clusters spans the majority of Scarborough. The other clusters are small and isolated near the south side of Scarborough.

### Cluster 1

In [173]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0,American Restaurant,Motel,Convenience Store,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


### Cluster 2

In [174]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,1,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
9,Scarborough,1,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Gym Pool,Gym,Grocery Store,Gas Station
4,Scarborough,1,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Gas Station,Vietnamese Restaurant,Electronics Store
13,Scarborough,1,Pharmacy,Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Fast Food Restaurant,Convenience Store,Fried Chicken Joint,Gas Station,Bank
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Brewery,Chinese Restaurant,Pet Store,Cosmetics Shop,Gym,Grocery Store,General Entertainment
7,Scarborough,1,Bakery,Ice Cream Shop,Bus Station,Metro Station,Soccer Field,Park,Intersection,Discount Store,Electronics Store,Fast Food Restaurant
2,Scarborough,1,Rental Car Location,Restaurant,Medical Center,Electronics Store,Breakfast Spot,Mexican Restaurant,Bank,Intersection,Gas Station,Cosmetics Shop
6,Scarborough,1,Discount Store,Hobby Shop,Bus Station,Coffee Shop,Department Store,Cosmetics Shop,Hakka Restaurant,Gym Pool,Gym,Grocery Store
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
14,Scarborough,1,Intersection,Playground,Park,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


### Cluster 3

In [175]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


### Cluster 4

In [176]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Convenience Store,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


### Cluster 5

In [177]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Playground,Vietnamese Restaurant,College Stadium,Gym Pool,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
